## Manual publication DB insertion from raw text using syntax features
### Publications and conferences of Dr. BOCICOR Maria Iuliana, Profesor Universitar
#### http://www.cs.ubbcluj.ro/~iuliana

In [1]:
class HelperMethods:
    @staticmethod
    def IsDate(text):
#         print("text")
#         print(text)
        for c in text.lstrip():
            if c not in "1234567890 ":
                return False
        return True

In [2]:
import pandas
import requests

In [3]:
page = requests.get('https://sites.google.com/view/iuliana-bocicor/research/publications')

In [4]:
data = page.text


In [5]:
from bs4 import BeautifulSoup

In [6]:
soup = BeautifulSoup(data)

In [142]:
def GetPublicationData_Number(text):
    title = text.split(',')[0].split('.')[1]
    try:
        date = [k.lstrip() for k in text.split(',') if HelperMethods.IsDate(k.lstrip())][0]
    except:
        date = ""
    return title, "", date

In [143]:
import re
def GetCoAuthorData(text):
#     print(text)
    val = re.search('\"[a-zA-Z ]+\"', text)
    title = val.group(0)
    val = re.search('Authors: [a-zA-Z,-. ]+ (?=Pages)', text)
    authors = val.group(0)
#     print(authors)
    return title, authors, ""

In [144]:
def GetPublicationData_A(text):
    print(text)
    print()
    text = text.replace("M. ", "")
    authors = text.split('.')[0]
    print("authors: ", authors)
    title = text.split('.')[1].lstrip(' \"')
    print("title: ", title)
    try:
        val = re.search('(19|20)[0-9]{2}\.', text)
        date = val.group(0).rstrip('.')
    except:
        date = ""
    
    print()
    
    
    return title, authors, date

In [145]:
pubs = []
# print(soup.find_all('div'))
for e in soup.find_all('div'):
    if "class" in e.attrs:
        if e.attrs["class"] == ["tyJCtd", "mGzaTb", "baZpAe"]:
            # for every pub entry
            for c in e.find_all("p", attrs={"class": "zfr3Q"}):
                if c.text == "":
                    continue
                if "co-author" in c.text:
                    rval = GetCoAuthorData(c.text)
                else:
                    features = c.text.split('.')
                    if features[0].isdecimal():
                        rval = GetPublicationData_Number(c.text)
                    else:
                        rval = GetPublicationData_A(c.text)
                pubs.append(rval)
                
 

Szuhai, Iulia-Monica, Maria Iuliana Bocicor, Emilia-Loredana Pop, Ioan-Gabriel Mircea, and Cristina Mircea. "Investigating Differences between Ancient and Modern Bacterial DNA in Dental Calculus via Autoencoders." In 2019 21st International Symposium on Symbolic and Numeric Algorithms for Scientific Computing (SYNASC), pp. 141-147. IEEE, 2019. 

authors:  Szuhai, Iulia-Monica, Maria Iuliana Bocicor, Emilia-Loredana Pop, Ioan-Gabriel Mircea, and Cristina Mircea
title:  Investigating Differences between Ancient and Modern Bacterial DNA in Dental Calculus via Autoencoders

Iuliana Marin, Maria-Iuliana Bocicor, Arthur-Jozsef Molnar. Indoor Localisation with Intelligent Luminaires for Home Monitoring. In Proceedings of the 14th International Conference on Evaluation of Novel Approaches to Software Engineering (ENASE 2019), pages 464-471, ISBN: 978-989-758-375-9, DOI: 10.5220/0007751304640471, 2019.

authors:  Iuliana Marin, Maria-Iuliana Bocicor, Arthur-Jozsef Molnar
title:  Indoor Localisa

In [146]:
for pub in pubs:
    print(pub)

(' Evaluation of Novel Approaches to Software Engineering', '', '2020')
('"Indoor Localization Techniques Within a Home Monitoring Platform"', 'Authors: Iuliana Marin, Maria Iuliana Bocicor, Arthur-Jozsef Molnar, ', '')
(' Evaluation of Novel Approaches to Software Engineering', '', '')
('"Technological Platform for the Prevention and Management of Healthcare Associated Infections and Outbreaks"', 'Authors: Maria Iuliana Bocicor, Maria Dascalu, Agnieszka Gaczowska, Sorin Hostiuc, Alin Moldoveanu, Antonio Molina, Arthur-Jozsef Molnar, Ionut Negoi, Vlad Racovita. ', '')
('Investigating Differences between Ancient and Modern Bacterial DNA in Dental Calculus via Autoencoders', 'Szuhai, Iulia-Monica, Maria Iuliana Bocicor, Emilia-Loredana Pop, Ioan-Gabriel Mircea, and Cristina Mircea', '2019')
('Indoor Localisation with Intelligent Luminaires for Home Monitoring', 'Iuliana Marin, Maria-Iuliana Bocicor, Arthur-Jozsef Molnar', '2019')
('Using clustering models for uncovering proteins’ structu

In [140]:
print("Count: ", len(pubs))

Count:  46


# DB Storage (TODO)
Time to store the entries in the `papers` DB table.


![Screenshot](Images/PapersTableSpec.PNG)

In [91]:
import mariadb
import json

In [92]:
with open('../credentials.json', 'r') as crd_json_fd:
    json_text = crd_json_fd.read()
    json_obj = json.loads(json_text)

In [93]:
credentials = json_obj["Credentials"]
username = credentials["username"]
password = credentials["password"]

In [94]:
table_name = "publications_cache"
db_name = "ubbcluj"

In [132]:
print(table_name)

publications_cache


In [95]:
mariadb_connection = mariadb.connect(user=username, password=password, database=db_name)
mariadb_cursor = mariadb_connection.cursor()

In [147]:
for paper in pubs:
    
    title = ""
    pub_date = ""
    authors = ""
    
    try:
        pub_date = paper[2].lstrip()
        pub_date = str(pub_date) + "-01-01"
        if len(pub_date) != 10:
            pub_date = ""
    except:
        pass
    
    try:
        title = paper[0].lstrip()
    except:
        pass
    
    try:
        authors = paper[1].lstrip()
    except AttributeError:
        pass
    
    
    
    insert_string = "INSERT INTO {0} SET ".format(table_name)
    insert_string += "Title=\'{0}\', ".format(title)
    insert_string += "ProfessorId=\'{0}\', ".format(7)
    if pub_date != "":
        insert_string += "PublicationDate=\'{0}\', ".format(str(pub_date))
    insert_string += "Authors=\'{0}\', ".format(authors)
    insert_string += "Affiliations=\'{0}\' ".format("")
    print(insert_string)
    

    try:
        mariadb_cursor.execute(insert_string)
    except mariadb.ProgrammingError as pe:
        print("Error")
        raise pe
    except mariadb.IntegrityError:
        continue

    

INSERT INTO publications_cache SET Title='Evaluation of Novel Approaches to Software Engineering', ProfessorId='7', PublicationDate='2020-01-01', Authors='', Affiliations='' 
INSERT INTO publications_cache SET Title='"Indoor Localization Techniques Within a Home Monitoring Platform"', ProfessorId='7', Authors='Authors: Iuliana Marin, Maria Iuliana Bocicor, Arthur-Jozsef Molnar, ', Affiliations='' 
INSERT INTO publications_cache SET Title='Evaluation of Novel Approaches to Software Engineering', ProfessorId='7', Authors='', Affiliations='' 
INSERT INTO publications_cache SET Title='"Technological Platform for the Prevention and Management of Healthcare Associated Infections and Outbreaks"', ProfessorId='7', Authors='Authors: Maria Iuliana Bocicor, Maria Dascalu, Agnieszka Gaczowska, Sorin Hostiuc, Alin Moldoveanu, Antonio Molina, Arthur-Jozsef Molnar, Ionut Negoi, Vlad Racovita. ', Affiliations='' 
INSERT INTO publications_cache SET Title='Investigating Differences between Ancient and M

In [148]:
mariadb_connection.close()

# Conclusion
### In the end, the DB only required ~2 manual modifications (deletions) with this code.
Out of the 46 papers, 32 were inserted with unique titles.

![Screenshot](Images/Finished.PNG)